# Loan Default Prediction - Classfication

## Part 4. Project Delievry

**This notebook contains script of**:
1. **premodeling functions**,
2. **model class**,
3. **final python file**.

---

<a id = 'toc'></a>
**Table of Contents**
1. [Premodeling Function](#clean)
    1. [load unseen data](#clean)
    2. [clean_data](#dc): 
        -[function ***clean_data***](#clean_func)
    3. [engineer_features](#fe): 
        -[function ***engineer_features***](#eng_func)
2. [Construct Model Class](#construct)
3. [Deploy Model](#deploy)
    1. [in Jupyter Notebook](#deploy)
    2. [Python script](#script)
        1. [Determine output format](#script)
        2. [Script](#scr)
            1. [How to execute](#exe)
            2. [Sample predictions](#predict)
    
---

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 150)
pd.set_option('max_rows', 180)

import pickle

<a id = 'clean'></a>
## 1. Premodeling Function
### 1.A. Load unseen data

In [2]:
df = pd.read_csv('../preds/open_loan_data_v20200309.csv')
df.shape

(64, 122)

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,accept_d,exp_d,list_d,credit_pull_d,review_status_d,review_status,url,desc,purpose,title,zip_code,addr_state,msa,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,delinq_amnt,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,mths_since_recent_inq,mths_since_recent_revol_delinq,mths_since_recent_bc,mort_acc,open_acc,pub_rec,total_bal_ex_mort,revol_bal,revol_util,total_bc_limit,total_acc,total_il_high_credit_limit,num_rev_accts,mths_since_recent_bc_dlq,initial_list_status,ils_exp_d,pub_rec_bankruptcies,num_accts_ever_120_pd,chargeoff_within_12_mths,collections_12_mths_ex_med,tax_liens,mths_since_last_major_derog,num_sats,num_tl_op_past_12m,mo_sin_rcnt_tl,tot_hi_cred_lim,tot_cur_bal,avg_cur_bal,num_bc_tl,num_actv_bc_tl,num_bc_sats,pct_tl_nvr_dlq,num_tl_90g_dpd_24m,num_tl_30dpd,num_tl_120dpd_2m,num_il_tl,mo_sin_old_il_acct,num_actv_rev_tl,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,total_rev_hi_lim,num_rev_tl_bal_gt_0,num_op_rev_tl,tot_coll_amt,effective_int_rate,application_type,disbursement_method,annual_inc_joint,dti_joint,verified_status_joint,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,mtg_payment,housing_payment,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,166274577,210605321,40000.0,23150.0,36,8.81,2.63,0.88,1268.46,A,A5,Sales Strategist,< 1 year,MORTGAGE,110000.0,Source Verified,02-05-2020 14:56:35,03-21-2020 15:56:36,03-01-2020 18:00:00,02-05-2020 14:38:05,03-01-2020 14:06:08,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,600xx,IL,NaN,0,3,3297,50.0,69.5,19.82,0,0.0,06-04-1994 17:00:00,665,669,1,69,70,4,69,47,1,11,1,49653,7503.0,69.5,10800,20,64382,12,69,f,NaN,1,0,0,0,0,NaN,8,1,6,75182,49653,6207,10,4,4,85,0,0,0,6,95,4,308,47,10800,4,7,0,8.79,INDIVIDUAL,CASH,,,,1,4,1,3,6,42150.0,65.5,0,0,2799.0,66.0,3,0,1,0.00,,,,,,,,,,,,,,
1,166375935,210889647,13825.0,6575.0,36,23.05,12.86,1.52,535.53,D,D3,Bus Operator,5 years,RENT,70000.0,Verified,02-19-2020 09:23:56,04-04-2020 10:23:57,03-08-2020 14:00:00,02-19-2020 09:15:32,03-08-2020 09:56:00,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,900xx,CA,NaN,0,4,718,50.0,77.6,14.64,0,0.0,08-18-2007 17:00:00,675,679,0,,35,16,,3,0,4,1,28071,2482.0,77.6,3200,11,28335,5,,f,NaN,1,0,0,0,0,NaN,4,2,3,31535,28071,7018,3,2,2,100,0,0,0,6,150,2,111,3,3200,2,2,0,22.25,INDIVIDUAL,CASH,,,,1,2,1,3,8,25589.0,90.3,1,1,1402.0,89.0,3,0,0,0.00,1100,,,,,,,,,,,,,
2,166325788,210728723,16800.0,12875.0,36,20.55,12.86,1.52,629.07,D,D2,NaN,NaN,OWN,44000.0,Verified,02-25-2020 12:54:16,03-29-2020 15:00:00,02-28-2020 14:00:00,02-06-2020 10:44:29,02-28-2020 09:55:15,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,752xx,TX,NaN,0,4,13650,0.0,21.1,17.73,1,0.0,10-05-2000 17:00:00,680,684,1,7,,3,7,3,3,22,0,7675,5286.0,11.6,17300,34,5000,26,,f,NaN,0,1,0,1,0,60.0,22,2,3,114155,9369,426,5,4,5,94,0,0,0,5,115,13,99,3,45600,13,20,1613,19.75,INDIVIDUAL,CASH,,,,1,1,0,1,18,2389.0,47.8,2,3,3388.0,15.2,1,0,1,849.93,,,,,,,,,,,,,,
3,167354641,212428141,40000.0,34775.0,36,6.46,2.63,0.88,1225.24,A,A1,NaN,< 1 year,MORTGAGE,100000.0,Not Verified,02-24-2020 20:01:19,03-29-2020 11:00:00,02-28-2020 10:00:00,02-24-2020 19:56:21,02-28-2020 05:52:20,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,945xx,CA,Na

In [4]:
df.dtypes.value_counts()

object     52
int64      51
float64    19
dtype: int64

[back to top](#toc)

<a id = 'dc'></a>
### 1.B. Clean data
   - [script for *clean_data* function](#clean_func)

In [5]:
orig_df = df.copy()

In [6]:
# Copied from p2-dataCleaning file
# drop: columns that exist in open loans but not in training data
cols_in_open_not_in_master = ['exp_default_rate',
                             'service_fee_rate',
                             'accept_d',
                             'exp_d',
                             'credit_pull_d',
                             'review_status_d',
                             'review_status',
                             'msa',
                             'ils_exp_d',
                             'effective_int_rate',
                             'disbursement_method',
                             'mtg_payment',
                             'housing_payment']
len(cols_in_open_not_in_master)

13

In [7]:
df = df.drop(cols_in_open_not_in_master, axis = 1)
df.shape

(64, 109)

In [8]:
# drop: columns that got dropped during training
drop_cols = ['id', 'url', 'emp_title', 'desc', 'title', 'member_id', 'initial_list_status','zip_code', 'addr_state',
            'funded_amnt']
len(drop_cols)

10

In [9]:
df = df.drop(drop_cols, axis = 1)
df.shape

(64, 99)

In [10]:
# 'revol_util', 'int_rate' are numerical features already, unlike in training data
df[['revol_util', 'int_rate']].describe()

,revol_util,int_rate
count,64.000000,64.000000
mean,55.001562,15.886875
std,22.101120,5.564681
min,0.000000,6.460000
25%,43.575000,11.710000
50%,58.450000,16.120000
75%,69.700000,20.550000
max,97.000000,28.800000


In [11]:
# convert 'grade', 'sub_grade' from categorical to ordinal feature
df[['grade', 'sub_grade']].describe()

,grade,sub_grade
count,64,64
unique,4,18
top,D,D2
freq,20,6


In [12]:
grade_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

In [13]:
df.loc[:, 'grade'] = df.grade.map(grade_map)
df.loc[:, 'sub_grade'] = df.sub_grade.apply(lambda x: grade_map[x[0]]*5 + int(x[1]))

In [14]:
df[['grade', 'sub_grade']].describe()

,grade,sub_grade
count,64.000000,64.000000
mean,1.765625,11.921875
std,1.034979,5.210809
min,0.000000,1.000000
25%,1.000000,8.000000
50%,2.000000,13.000000
75%,3.000000,17.000000
max,3.000000,20.000000


In [15]:
# convert 'emp_length' from categorical to ordinal feature
el_map = {'10+ years': 10.0,
         '2 years': 2.0,
         '< 1 year': 0.5,
         '3 years': 3.0,
         '1 year': 1.0,
         '5 years': 5.0,
         '4 years': 4.0,
         '6 years': 6.0,
         '7 years': 7.0,
         '8 years': 8.0,
         '9 years': 9.0}

In [16]:
df.emp_length.describe()

count           57
unique          11
top       < 1 year
freq            16
Name: emp_length, dtype: object

In [17]:
df.loc[:, 'emp_length'] = df.emp_length.apply(lambda x: el_map[x] if pd.notnull(x) else 0)

In [18]:
df.emp_length.describe()

count    64.00000
mean      4.21875
std       3.97799
min       0.00000
25%       0.50000
50%       2.50000
75%       8.25000
max      10.00000
Name: emp_length, dtype: float64

In [19]:
# convert 'list_d', 'earliest_cr_line', 'sec_app_earliest_cr_line' from categorical to timedate features
df[['list_d', 'earliest_cr_line', 'sec_app_earliest_cr_line']].describe()

,list_d,earliest_cr_line,sec_app_earliest_cr_line
count,64,64,64
unique,34,64,16
top,03-06-2020 10:00:00,03-03-2004 16:00:00,
freq,5,1,49


In [20]:
df.loc[:, 'sec_app_earliest_cr_line'] = pd.to_datetime(df.sec_app_earliest_cr_line, errors='coerce')
df.loc[:, 'earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, errors = 'coerce')
df.loc[:, 'list_d'] = pd.to_datetime(df.list_d, errors = 'coerce')

In [21]:
df[['list_d', 'earliest_cr_line', 'sec_app_earliest_cr_line']].describe()

,list_d,earliest_cr_line,sec_app_earliest_cr_line
count,64,64,15
unique,34,64,15
top,2020-03-06 10:00:00,2012-06-28 17:00:00,1997-01-03 16:00:00
freq,5,1,1
first,2020-02-26 18:00:00,1980-11-08 16:00:00,1989-03-02 16:00:00
last,2020-03-09 14:00:00,2015-05-08 17:00:00,2014-08-07 17:00:00


In [22]:
df.home_ownership.value_counts()

MORTGAGE    28
RENT        26
OWN         10
Name: home_ownership, dtype: int64

In [23]:
# group home_ownership feature's 'NONE', 'OTHER' value into 'ANY'
df.loc[:, 'home_ownership'] = df.home_ownership.replace(['NONE', 'OTHER'], 'ANY')

In [24]:
# categorical columns copied from p2-dataClearning file 
master_cat_cols = ['term',
                    'home_ownership',
                    'is_inc_v',
                    'purpose',
                    'application_type',
                    'verified_status_joint']

In [25]:
for col in master_cat_cols:
    print (df[col].value_counts(), '\n')

36    55
60     9
Name: term, dtype: int64 

MORTGAGE    28
RENT        26
OWN         10
Name: home_ownership, dtype: int64 

Not Verified       38
Source Verified    20
Verified            6
Name: is_inc_v, dtype: int64 

Debt consolidation         30
Credit card refinancing    19
Other                       6
Home improvement            3
Business                    2
Car financing               2
Home buying                 1
Moving and relocation       1
Name: purpose, dtype: int64 

INDIVIDUAL    49
JOINT         15
Name: application_type, dtype: int64 

                   49
Not Verified       12
Source Verified     2
Verified            1
Name: verified_status_joint, dtype: int64 



In [26]:
# 'purpose' feature in open loan file has differnet values compared to training data
# replace these with values in training data
df.loc[:, 'purpose'] = df.purpose.apply(lambda x: x.replace(" ", "_").lower())

In [27]:
df.purpose.value_counts()

debt_consolidation         30
credit_card_refinancing    19
other                       6
home_improvement            3
business                    2
car_financing               2
home_buying                 1
moving_and_relocation       1
Name: purpose, dtype: int64

In [28]:
purpose_map = {'credit_card_refinancing':'credit_card',
              'home_buying':'house',
              'business':'small_business',
              'moving_and_relocation':'moving',
              'car_financing': 'car'}
df.loc[:, 'purpose'] = df.purpose.replace(purpose_map)

In [29]:
df.purpose.value_counts()

debt_consolidation    30
credit_card           19
other                  6
home_improvement       3
car                    2
small_business         2
moving                 1
house                  1
Name: purpose, dtype: int64

In [30]:
df.term.value_counts()

36    55
60     9
Name: term, dtype: int64

In [31]:
# 'term' is a numerical feature in unseen data vs. categorical in training 
# convert from numerical to categorical to be consistent
df.loc[:, 'term'] = df.term.replace({36: ' 36 months', 60: ' 60 months'})
df.term.value_counts()

 36 months    55
 60 months     9
Name: term, dtype: int64

In [32]:
df.application_type.value_counts()

INDIVIDUAL    49
JOINT         15
Name: application_type, dtype: int64

In [33]:
# 'application_type' feature in unseen data has differnet values vs. training data
# replace these with values in training data
df.loc[:, 'application_type'] = df.application_type.replace({'INDIVIDUAL':'Individual', 'JOINT':'Joint App'})
df.application_type.value_counts()

Individual    49
Joint App     15
Name: application_type, dtype: int64

In [34]:
cat_cols = df.dtypes[df.dtypes == 'object'].index
cat_cols

Index(['term', 'home_ownership', 'is_inc_v', 'purpose', 'bc_open_to_buy',
       'percent_bc_gt_75', 'bc_util', 'mths_since_last_delinq',
       'mths_since_last_record', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'mths_since_recent_bc',
       'mths_since_recent_bc_dlq', 'num_tl_120dpd_2m', 'mo_sin_old_il_acct',
       'application_type', 'annual_inc_joint', 'dti_joint',
       'verified_status_joint', 'mths_since_rcnt_il', 'il_util',
       'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
       'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog'],
      dtype='object')

In [35]:
df.dtypes.value_counts()

int64             50
object            33
float64           13
datetime64[ns]     3
dtype: int64

In [36]:
df[cat_cols].describe().transpose()

,count,unique,top,freq
term,64,2,36 months,55
home_ownership,64,3,MORTGAGE,28
is_inc_v,64,3,Not Verified,38
purpose,64,8,debt_consolidation,30
bc_open_to_buy,64,64,1286,1
percent_bc_gt_75,64,22,50.0,11
bc_util,64,60,69.5,3
mths_since_last_delinq,64,29,,28
mths_since_last_record,64,9,,55
mths_since_recent_inq,64,21,4,11


In [37]:
# many features in open loan data are categorical instead of numerical as in training file,
# convert these into numerical features
cat_to_num_cols = [col for col in cat_cols if col not in master_cat_cols]
len(cat_to_num_cols)

27

In [38]:
cat_to_num_cols

['bc_open_to_buy',
 'percent_bc_gt_75',
 'bc_util',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'num_tl_120dpd_2m',
 'mo_sin_old_il_acct',
 'annual_inc_joint',
 'dti_joint',
 'mths_since_rcnt_il',
 'il_util',
 'revol_bal_joint',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog']

In [39]:
[col for col in master_cat_cols if col in cat_cols]

['term',
 'home_ownership',
 'is_inc_v',
 'purpose',
 'application_type',
 'verified_status_joint']

In [40]:
# convert some categorical features to numerical, if missing, then, fill with np.NAN
df.loc[:, cat_to_num_cols] = df[cat_to_num_cols].apply(lambda s: pd.to_numeric(s, errors = 'coerce'))

In [41]:
df.dtypes.value_counts()

int64             50
float64           40
object             6
datetime64[ns]     3
dtype: int64

In [42]:
df.dtypes[df.dtypes == 'object'].index

Index(['term', 'home_ownership', 'is_inc_v', 'purpose', 'application_type',
       'verified_status_joint'],
      dtype='object')

In [43]:
# 'verified_status_joint' feature's missing values are filled with ' ' rather an np.NaN,
# replace with 'Missing' to be consistent with training data
df.verified_status_joint.value_counts()

                   49
Not Verified       12
Source Verified     2
Verified            1
Name: verified_status_joint, dtype: int64

In [44]:
df.loc[:, master_cat_cols] = df[master_cat_cols].fillna('Missing')

In [45]:
df.verified_status_joint.value_counts()

                   49
Not Verified       12
Source Verified     2
Verified            1
Name: verified_status_joint, dtype: int64

In [46]:
df.loc[:, 'verified_status_joint'] = df.verified_status_joint.replace({' ': 'Missing'})
df.verified_status_joint.value_counts()

Missing            49
Not Verified       12
Source Verified     2
Verified            1
Name: verified_status_joint, dtype: int64

In [47]:
df[master_cat_cols].isnull().sum().sum()

0

In [48]:
# to impute numerical columns with missing value, fill it with heuristics developed from training data
with open('num_cols_miss_imputing_dict.pkl','rb') as f:
    num_cols_miss_imputing_dict = pickle.load(f)

In [49]:
num_cols_miss_imputing_dict['num_cols_miss_median_values']

dti                               17.24
revol_util                        53.70
collections_12_mths_ex_med         0.00
tot_coll_amt                       0.00
tot_cur_bal                    71754.00
open_acc_6m                        1.00
open_act_il                        2.00
open_il_12m                        0.00
open_il_24m                        1.00
mths_since_rcnt_il                12.00
total_bal_il                   22357.00
il_util                           74.00
open_rv_12m                        1.00
open_rv_24m                        2.00
max_bal_bc                      3937.00
all_util                          60.00
total_rev_hi_lim               22800.00
inq_fi                             1.00
total_cu_tl                        0.00
inq_last_12m                       2.00
acc_open_past_24mths               4.00
avg_cur_bal                     6820.00
bc_open_to_buy                  4180.00
bc_util                           65.50
chargeoff_within_12_mths           0.00


In [50]:
for k, v in num_cols_miss_imputing_dict.items():
    print(k, len(v))

num_cols_miss_median 54
num_cols_miss 59
num_cols_miss_1200 5
num_cols_miss_median_values 54
num_cols_miss_sparse 15


In [51]:
df.shape

(64, 99)

In [52]:
# drop columns deemed sparsed thus dropped in training
df = df.drop(num_cols_miss_imputing_dict['num_cols_miss_sparse'], axis = 1)
df.shape

(64, 84)

In [53]:
for col in num_cols_miss_imputing_dict['num_cols_miss']:
    df.loc[:, col+'_mflag'] = df[col].isnull()

In [54]:
df.shape

(64, 143)

In [55]:
# impute columns, whose missing value signifies a lack of adverse events, with large values 
# because large values is better (1200 = 12 months * 100 years)
df[num_cols_miss_imputing_dict['num_cols_miss_1200']].describe()

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq
count,36.000000,9.000000,19.000000,21.000000,31.000000
mean,33.138889,74.777778,47.368421,36.952381,34.548387
std,23.245464,22.719913,22.857555,23.911245,23.292114
min,4.000000,35.000000,8.000000,4.000000,4.000000
25%,11.250000,68.000000,31.500000,16.000000,13.000000
50%,36.500000,70.000000,49.000000,39.000000,37.000000
75%,50.250000,81.000000,61.500000,52.000000,51.000000
max,80.000000,110.000000,80.000000,80.000000,80.000000


In [56]:
for col in num_cols_miss_imputing_dict['num_cols_miss_1200']:
    df.loc[:, col] = df[col].fillna(1200)

In [57]:
df[num_cols_miss_imputing_dict['num_cols_miss_1200']].describe()

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_recent_bc_dlq,mths_since_recent_revol_delinq
count,64.000000,64.000000,64.000000,64.000000,64.000000
mean,543.640625,1041.765625,857.812500,818.375000,635.484375
std,583.687766,394.341233,530.919086,550.568263,587.265549
min,4.000000,35.000000,8.000000,4.000000,4.000000
25%,20.750000,1200.000000,74.000000,54.250000,37.750000
50%,65.000000,1200.000000,1200.000000,1200.000000,1200.000000
75%,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
max,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000


In [58]:
# impute features with median as missing values if the reason for missing is not available, 
# unclear, or couldn't be justified with extreme small or large values
df[num_cols_miss_imputing_dict['num_cols_miss_median']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dti,64.0,28.441406,37.708710,0.0,14.5325,23.75,31.32,305.33
revol_util,64.0,55.001562,22.101120,0.0,43.5750,58.45,69.70,97.00
collections_12_mths_ex_med,64.0,0.062500,0.243975,0.0,0.0000,0.00,0.00,1.00
tot_coll_amt,64.0,42.390625,210.468760,0.0,0.0000,0.00,0.00,1613.00
tot_cur_bal,64.0,178730.656250,244738.889114,0.0,33962.0000,86728.50,236801.25,1290808.00
open_acc_6m,64.0,0.859375,0.957298,0.0,0.0000,1.00,1.00,4.00
open_act_il,64.0,3.093750,3.575573,0.0,1.0000,2.00,3.00,24.00
open_il_12m,64.0,0.562500,0.833333,0.0,0.0000,0.00,1.00,3.00
open_il_24m,64.0,1.593750,1.305893,0.0,1.0000,1.00,3.00,6.00
mths_since_rcnt_il,61.0,16.852459,12.414825,1.0,8.0000,14.00,21.00,57.00


In [59]:
# checking to make sure medians are within the range of unseen data
pd.concat([df[num_cols_miss_imputing_dict['num_cols_miss_median']].describe().transpose(),
           num_cols_miss_imputing_dict['num_cols_miss_median_values']], axis = 1)

,count,mean,std,min,25%,50%,75%,max,0
dti,64.0,28.441406,37.708710,0.0,14.5325,23.75,31.32,305.33,17.24
revol_util,64.0,55.001562,22.101120,0.0,43.5750,58.45,69.70,97.00,53.70
collections_12_mths_ex_med,64.0,0.062500,0.243975,0.0,0.0000,0.00,0.00,1.00,0.00
tot_coll_amt,64.0,42.390625,210.468760,0.0,0.0000,0.00,0.00,1613.00,0.00
tot_cur_bal,64.0,178730.656250,244738.889114,0.0,33962.0000,86728.50,236801.25,1290808.00,71754.00
open_acc_6m,64.0,0.859375,0.957298,0.0,0.0000,1.00,1.00,4.00,1.00
open_act_il,64.0,3.093750,3.575573,0.0,1.0000,2.00,3.00,24.00,2.00
open_il_12m,64.0,0.562500,0.833333,0.0,0.0000,0.00,1.00,3.00,0.00
open_il_24m,64.0,1.593750,1.305893,0.0,1.0000,1.00,3.00,6.00,1.00
mths_since_rcnt_il,61.0,16.852459,12.414825,1.0,8.0000,14.00,21.00,57.00,12.00


In [60]:
for col in num_cols_miss_imputing_dict['num_cols_miss_median']:
    df.loc[:, col] = df[col].fillna(num_cols_miss_imputing_dict['num_cols_miss_median_values'][col])

In [61]:
df[num_cols_miss_imputing_dict['num_cols_miss_median']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dti,64.0,28.441406,37.708710,0.0,14.5325,23.75,31.320,305.33
revol_util,64.0,55.001562,22.101120,0.0,43.5750,58.45,69.700,97.00
collections_12_mths_ex_med,64.0,0.062500,0.243975,0.0,0.0000,0.00,0.000,1.00
tot_coll_amt,64.0,42.390625,210.468760,0.0,0.0000,0.00,0.000,1613.00
tot_cur_bal,64.0,178730.656250,244738.889114,0.0,33962.0000,86728.50,236801.250,1290808.00
open_acc_6m,64.0,0.859375,0.957298,0.0,0.0000,1.00,1.000,4.00
open_act_il,64.0,3.093750,3.575573,0.0,1.0000,2.00,3.000,24.00
open_il_12m,64.0,0.562500,0.833333,0.0,0.0000,0.00,1.000,3.00
open_il_24m,64.0,1.593750,1.305893,0.0,1.0000,1.00,3.000,6.00
mths_since_rcnt_il,64.0,16.625000,12.159652,1.0,8.7500,14.00,20.250,57.00


In [62]:
df.isnull().sum().sum()

0

In [63]:
df.shape

(64, 143)

**[back to top](#toc)**

<a id = 'clean_func'></a>
#### Create *clean_data* function

In [64]:
# information to be pickled for use in future data cleaning
drop_cols_in_open_not_in_master = ['exp_default_rate', 'service_fee_rate', 'accept_d', 'exp_d', 'credit_pull_d', 'review_status_d', 'review_status', 'msa', 'ils_exp_d', 'effective_int_rate', 'disbursement_method', 'mtg_payment', 'housing_payment']

drop_cols = ['id', 'url', 'emp_title', 'desc', 'title', 'member_id', 'initial_list_status','zip_code', 'addr_state', 'funded_amnt']

master_cat_cols = ['term', 'home_ownership', 'is_inc_v', 'purpose', 'application_type', 'verified_status_joint']

cat_to_num_cols = ['bc_open_to_buy','percent_bc_gt_75', 'bc_util', 'mths_since_last_delinq', 'mths_since_last_record', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'num_tl_120dpd_2m', 'mo_sin_old_il_acct', 'annual_inc_joint', 'dti_joint', 'mths_since_rcnt_il', 'il_util', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog']

grade_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

el_map = {'10+ years': 10.0,'2 years': 2.0, '< 1 year': 0.5,'3 years': 3.0, '1 year': 1.0,'5 years': 5.0, '4 years': 4.0, '6 years': 6.0, '7 years': 7.0, '8 years': 8.0, '9 years': 9.0}

purpose_map = {'credit_card_refinancing':'credit_card', 
               'home_buying':'house', 
               'business':'small_business', 
               'moving_and_relocation':'moving',
              'car_financing': 'car'}

In [65]:
clean_dict = {'drop_cols_in_open_not_in_master': drop_cols_in_open_not_in_master,
              'drop_cols': drop_cols,
              'cat_to_num_cols': cat_to_num_cols,
              'cat_cols': master_cat_cols,
              'grade_map': grade_map,
              'el_map': el_map,
             'purpose_map': purpose_map}

In [66]:
clean_dict = {**clean_dict, **num_cols_miss_imputing_dict}
len(clean_dict)

12

In [67]:
for k in clean_dict.keys():
    print (k)

drop_cols_in_open_not_in_master
drop_cols
cat_to_num_cols
cat_cols
grade_map
el_map
purpose_map
num_cols_miss_median
num_cols_miss
num_cols_miss_1200
num_cols_miss_median_values
num_cols_miss_sparse


In [68]:
def clean_data (df, clean_dict):

    df = df.drop(clean_dict['drop_cols_in_open_not_in_master'], axis = 1)
    df = df.drop(clean_dict['drop_cols'], axis = 1)
    
    df.loc[:, 'grade'] = df.grade.map(clean_dict['grade_map'])
    df.loc[:, 'sub_grade'] = df.sub_grade.apply(lambda x: clean_dict['grade_map'][x[0]]*5 + int(x[1]))
    df.loc[:, 'emp_length'] = df.emp_length.apply(lambda x: clean_dict['el_map'][x] if pd.notnull(x) else 0)
    df.loc[:, 'sec_app_earliest_cr_line'] = pd.to_datetime(df.sec_app_earliest_cr_line, errors='coerce')
    df.loc[:, 'earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, errors = 'coerce')
    df.loc[:, 'list_d'] = pd.to_datetime(df.list_d, errors = 'coerce')
    df.loc[:, 'home_ownership'] = df.home_ownership.replace(['NONE', 'OTHER'], 'ANY')
    df.loc[:, 'purpose'] = df.purpose.apply(lambda x: x.replace(" ", "_").lower())
    try:
        df.loc[:, 'purpose'] = df.purpose.replace(clean_dict['purpose_map'])
    except:
        print("Error with categorical feature purpose:", sys.exc_info()[0])
        raise
    df.loc[:, 'term'] = df.term.replace({36: ' 36 months', 60: ' 60 months'})
    df.loc[:, 'application_type'] = df.application_type.replace({'INDIVIDUAL':'Individual', 'JOINT':'Joint App'})

    df.loc[:, clean_dict['cat_to_num_cols']] = df[clean_dict['cat_to_num_cols']].apply(lambda s: pd.to_numeric(s, errors = 'coerce'))
    df.loc[:, clean_dict['cat_cols']] = df[clean_dict['cat_cols']].fillna('Missing')
    df.loc[:, 'verified_status_joint'] = df.verified_status_joint.replace({' ': 'Missing'})
    df = df.drop(clean_dict['num_cols_miss_sparse'], axis = 1)
    for col in clean_dict['num_cols_miss']:
        df.loc[:, col+'_mflag'] = df[col].isnull()
    for col in clean_dict['num_cols_miss_1200']:
        df.loc[:, col] = df[col].fillna(1200)
    for col in clean_dict['num_cols_miss_median']:
        df.loc[:, col] = df[col].fillna(clean_dict['num_cols_miss_median_values'][col])
        
    return df

In [69]:
cleaned_df = clean_data(orig_df, clean_dict)

In [70]:
# checking accuracy of the clean_data function
(cleaned_df == df).all().all()

True

**[back to top](#toc)**

<a id = 'fe'></a>
### 1.C. Engineer features

In [71]:
df[['list_d', 'earliest_cr_line']].head()

,list_d,earliest_cr_line
0,2020-03-01 18:00:00,1994-06-04 17:00:00
1,2020-03-08 14:00:00,2007-08-18 17:00:00
2,2020-02-28 14:00:00,2000-10-05 17:00:00
3,2020-02-28 10:00:00,2002-04-24 17:00:00
4,2020-03-02 18:00:00,1999-07-01 17:00:00


In [72]:
df.loc[:, 'credit_history'] = ((df.list_d - df.earliest_cr_line)/ np.timedelta64(1, 'D')).astype(int)
df.credit_history.head()

0    9402
1    4585
2    7084
3    6518
4    7550
Name: credit_history, dtype: int64

In [73]:
df = df.drop(['earliest_cr_line','list_d'], axis = 1)
df.shape

(64, 142)

In [74]:
df[['installment', 'annual_inc']].head()

,installment,annual_inc
0,1268.46,110000.0
1,535.53,70000.0
2,629.07,44000.0
3,1225.24,100000.0
4,1029.71,93000.0


In [75]:
# itlm = income to loan payment multiples
df.loc[:, 'itlm'] = df.annual_inc/df.installment
df.itlm.head()

0     86.719329
1    130.711631
2     69.944521
3     81.616663
4     90.316691
Name: itlm, dtype: float64

In [76]:
df.dtypes.sort_values().value_counts()

bool       59
int64      51
float64    27
object      6
dtype: int64

In [77]:
df.shape

(64, 143)

In [78]:
# one hot encoding
df = pd.get_dummies(df)

**Align test column with train column**

In [79]:
test_df = pd.read_csv('../data/abt/analytical_base_table-test.csv')

In [80]:
# columns from training data
abt_cols = test_df.columns.tolist()

In [81]:
# columns from open loans
open_cols = df.columns.tolist()

In [82]:
[col for col in open_cols if col not in abt_cols]

[]

In [83]:
# it is normal that certian catogories are not present in the open loans data
[col for col in abt_cols if col not in open_cols]

['loan_status',
 'home_ownership_ANY',
 'purpose_educational',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_renewable_energy',
 'purpose_vacation',
 'purpose_wedding']

In [84]:
test_df = test_df.drop('loan_status', axis = 1)

In [85]:
test_df.shape

(102911, 166)

In [86]:
abt_df = test_df.head(0)
abt_df.shape

(0, 166)

In [87]:
abt_df

,loan_amnt,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,...,open_rv_12m_mflag,open_rv_24m_mflag,max_bal_bc_mflag,all_util_mflag,total_rev_hi_lim_mflag,inq_fi_mflag,total_cu_tl_mflag,inq_last_12m_mflag,acc_open_past_24mths_mflag,avg_cur_bal_mflag,bc_open_to_buy_mflag,bc_util_mflag,chargeoff_within_12_mths_mflag,mo_sin_old_il_acct_mflag,mo_sin_old_rev_tl_op_mflag,mo_sin_rcnt_rev_tl_op_mflag,mo_sin_rcnt_tl_mflag,mort_acc_mflag,mths_since_recent_bc_mflag,mths_since_recent_bc_dlq_mflag,mths_since_recent_inq_mflag,mths_since_recent_revol_delinq_mflag,num_accts_ever_120_pd_mflag,num_actv_bc_tl_mflag,num_actv_rev_tl_mflag,num_bc_sats_mflag,num_bc_tl_mflag,num_il_tl_mflag,num_op_rev_tl_mflag,num_rev_accts_mflag,num_rev_tl_bal_gt_0_mflag,num_sats_mflag,num_tl_120dpd_2m_mflag,num_tl_30dpd_mflag,num_tl_90g_dpd_24m_mflag,num_tl_op_past_12m_mflag,pct_tl_nvr_dlq_mflag,percent_bc_gt_75_mflag,pub_rec_bankruptcies_mflag,tax_liens_mflag,tot_hi_cred_lim_mflag,total_bal_ex_mort_mflag,total_bc_limit_mflag,total_il_high_credit_limit_mflag,credit_history,itlm,term_ 36 months,term_ 60 months,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,is_inc_v_Not Verified,is_inc_v_Source Verified,is_inc_v_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Individual,application_type_Joint App,verified_status_joint_Missing,verified_status_joint_Not Verified,verified_status_joint_Source Verified,verified_status_joint_Verified


In [88]:
# align columns
df, _ = df.align(abt_df, join = 'right', axis = 1, fill_value=0)

In [89]:
open_cols = df.columns.tolist()

In [90]:
[col for col in test_df.columns.tolist() if col not in open_cols]

[]

In [91]:
df.shape

(64, 166)

**[back to top](#toc)**

<a id = 'eng_func'></a>
#### Create *engineer_features* function

In [92]:
engineer_dict = {'abt_df': abt_df}

In [93]:
def engineer_features(df, engineer_dict):
    
    df.loc[:, 'credit_history'] = ((df.list_d - df.earliest_cr_line)/ np.timedelta64(1, 'D')).astype(int)
    df = df.drop(['earliest_cr_line','list_d'], axis = 1)
    df.loc[:, 'itlm'] = df.annual_inc/df.installment
    df = pd.get_dummies(df)
    df, _ = df.align(engineer_dict['abt_df'], join = 'right', axis = 1, fill_value = 0)   
    
    return df    
    

In [94]:
augmented_data = engineer_features(cleaned_df, engineer_dict)

In [95]:
(augmented_data == df).all().all()

True

**[back to top](#toc)**

<a id = 'construct'></a>
### 2. Construct model class

In [96]:
# Load fitted model
with open('fitted_models.pkl','rb') as f:
    fitted_models = pickle.load(f)

In [97]:
fitted_model_lr = fitted_models['lr'].best_estimator_
fitted_model_lr

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=0.001, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=-1, penalty='l2',
                                    random_state=26, solver='lbfgs', tol=0.0001,
                                    verbose=0, warm_start=False))],
         verbose=False)

In [98]:
class LoanDefaultModel:
    """A model that takes loan data from LendingClub.com and predicts loan default rate."""
    
    def __init__(self, model_dict_location):
        with open(model_dict_location, 'rb') as f:
            model_dict = pickle.load(f)
            self.model = model_dict['model']
            self.clean_dict = model_dict['clean_dict']
            self.engineer_dict = model_dict['engineer_dict']
            self.threshold_dict = model_dict['threshold_dict']
    
    def predict_proba(self, X, clean=True, augment = True):
        X_orig = X.copy()
        if clean:
            X = self.clean_data(X)
        if augment:
            X = self.engineer_features(X)
        return X_orig, X, self.model.predict_proba(X)[:,1]
    
    def clean_data(self, df):        
        clean_dict = self.clean_dict
        
        df = df.drop(clean_dict['drop_cols_in_open_not_in_master'], axis = 1)
        df = df.drop(clean_dict['drop_cols'], axis = 1)

        df.loc[:, 'grade'] = df.grade.map(clean_dict['grade_map'])
        df.loc[:, 'sub_grade'] = df.sub_grade.apply(lambda x: clean_dict['grade_map'][x[0]]*5 + int(x[1]))
        df.loc[:, 'emp_length'] = df.emp_length.apply(lambda x: clean_dict['el_map'][x] if pd.notnull(x) else 0)
        df.loc[:, 'sec_app_earliest_cr_line'] = pd.to_datetime(df.sec_app_earliest_cr_line, errors='coerce')
        df.loc[:, 'earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, errors = 'coerce')
        df.loc[:, 'list_d'] = pd.to_datetime(df.list_d, errors = 'coerce')
        df.loc[:, 'home_ownership'] = df.home_ownership.replace(['NONE', 'OTHER'], 'ANY')
        df.loc[:, 'purpose'] = df.purpose.apply(lambda x: x.replace(" ", "_").lower())
        try:
            df.loc[:, 'purpose'] = df.purpose.replace(clean_dict['purpose_map'])
        except:
            print("Error with categorical feature purpose:", sys.exc_info()[0])
            raise
        df.loc[:, 'term'] = df.term.replace({36: ' 36 months', 60: ' 60 months'})
        df.loc[:, 'application_type'] = df.application_type.replace({'INDIVIDUAL':'Individual', 'JOINT':'Joint App'})

        df.loc[:, clean_dict['cat_to_num_cols']] = df[clean_dict['cat_to_num_cols']].apply(lambda s: pd.to_numeric(s, errors = 'coerce'))
        df.loc[:, clean_dict['cat_cols']] = df[clean_dict['cat_cols']].fillna('Missing')
        df.loc[:, 'verified_status_joint'] = df.verified_status_joint.replace({' ': 'Missing'})
        df = df.drop(clean_dict['num_cols_miss_sparse'], axis = 1)
        for col in clean_dict['num_cols_miss']:
            df.loc[:, col+'_mflag'] = df[col].isnull()
        for col in clean_dict['num_cols_miss_1200']:
            df.loc[:, col] = df[col].fillna(1200)
        for col in clean_dict['num_cols_miss_median']:
            df.loc[:, col] = df[col].fillna(clean_dict['num_cols_miss_median_values'][col])

        return df
    
    
    def engineer_features(self, df):
        engineer_dict = self.engineer_dict
        
        df.loc[:, 'credit_history'] = ((df.list_d - df.earliest_cr_line)/ np.timedelta64(1, 'D')).astype(int)
        df = df.drop(['earliest_cr_line','list_d'], axis = 1)
        df.loc[:, 'itlm'] = df.annual_inc/df.installment
        df = pd.get_dummies(df)
        df, _ = df.align(engineer_dict['abt_df'], join = 'right', axis = 1, fill_value = 0)   

        return df 
    

**[back to top](#toc)**

<a id = 'deploy'></a>
### 3. Model deployment
#### 3.A. Jupyter notebook

In [99]:
# saved from p3-modelSelection
with open('threshold_dict.pkl', 'rb') as f:
    threshold_dict = pickle.load(f)

In [100]:
for k, v, in threshold_dict.items():
    print (k)
    print (v, '\n')

thres_05
0.06410193764508962 

thres_05_df
            fpr       tpr  thresholds       edr      ptl
14318  0.903362  0.979965    0.064102  0.049998  0.08113 

thres_08
0.10968430507396774 

thres_08_df
            fpr       tpr  thresholds       edr       ptl
12924  0.710754  0.900965    0.109684  0.079965  0.250737 

thres_10
0.13741758307667645 

thres_10_df
            fpr       tpr  thresholds       edr       ptl
11459  0.574702  0.813869    0.137418  0.099989  0.376877 



In [101]:
model_dict = {'model': fitted_model_lr,
             'clean_dict': clean_dict,
             'engineer_dict': engineer_dict,
             'threshold_dict': threshold_dict}

In [267]:
with open('model_dict.pkl', 'wb') as f:
    pickle.dump(model_dict, f)

In [102]:
with open('model_dict.pkl', 'rb') as f:
    saved_model_dict = pickle.load(f)

In [103]:
for k, v, in saved_model_dict.items():
    print (k, type(v))

model <class 'sklearn.pipeline.Pipeline'>
clean_dict <class 'dict'>
engineer_dict <class 'dict'>
threshold_dict <class 'dict'>


**Checking whether model is properly saved**

In [104]:
abt_az = pd.read_csv('../data/abt/analytical_base_table-analyze.csv') 
abt_az.shape

(50000, 167)

In [105]:
y_az = abt_az.loan_status
X_az = abt_az.drop('loan_status', axis = 1)

In [106]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_az, saved_model_dict['model'].predict_proba(X_az)[:,1])

0.6778323956079385

In [107]:
roc_auc_score(y_az, fitted_model_lr.predict_proba(X_az)[:,1])

0.6778323956079385

In [108]:
# initialze LoanDefaultModel instance
model = LoanDefaultModel('model_dict.pkl')

In [109]:
# make predictions
_, _, pred = model.predict_proba(df, False, False)

In [110]:
pred.shape

(64,)

In [111]:
pred[:20]

array([0.15996257, 0.28173983, 0.30271524, 0.00848962, 0.13096513,
       0.24174724, 0.07634736, 0.17561899, 0.15818549, 0.15662897,
       0.35581144, 0.16255377, 0.2754162 , 0.04097623, 0.17737659,
       0.15888887, 0.07997987, 0.26537516, 0.37557246, 0.04293048])

In [112]:
fitted_model_lr.predict_proba(df)[:,1][:20]

array([0.15996257, 0.28173983, 0.30271524, 0.00848962, 0.13096513,
       0.24174724, 0.07634736, 0.17561899, 0.15818549, 0.15662897,
       0.35581144, 0.16255377, 0.2754162 , 0.04097623, 0.17737659,
       0.15888887, 0.07997987, 0.26537516, 0.37557246, 0.04293048])

In [113]:
# make predictions on cleaned data
_, _, pred_cd = model.predict_proba(cleaned_df, False, True)

In [114]:
# make predictions on augmented data
_, _, pred_ef = model.predict_proba(augmented_data, False, False)

In [115]:
# make predictions on original data
_, _, pred_cd_ef = model.predict_proba(orig_df, True, True)

In [116]:
# check the results are the same
print(np.array_equal(pred_cd, pred_ef),
      np.array_equal(pred_ef, pred_cd_ef),
      np.array_equal(pred, pred_cd_ef))

True False False


**Note**:
Final notebook p5-predictDefault

**[back to top](#toc)**

<a id = 'script'></a>
#### 3.B. Python script
#### 3.B.a. Determine prediction output format

In [117]:
orig_df.shape

(64, 122)

In [118]:
orig_df.loc[:, 'predict_proba'] = fitted_model_lr.predict_proba(df)[:,1]

In [119]:
orig_df.loc[:, 'gr_than_thres_05'] = (orig_df.predict_proba > threshold_dict['thres_05'])
orig_df.loc[:, 'gr_than_thres_08'] = (orig_df.predict_proba > threshold_dict['thres_08'])
orig_df.loc[:, 'gr_than_thres_10'] = (orig_df.predict_proba > threshold_dict['thres_10'])

In [120]:
# final output format
orig_df[['id','sub_grade', 'int_rate','term','loan_amnt','exp_default_rate', 'predict_proba', 
         'gr_than_thres_05', 'gr_than_thres_08', 'gr_than_thres_10']].sort_values(['sub_grade']).head()

,id,sub_grade,int_rate,term,loan_amnt,exp_default_rate,predict_proba,gr_than_thres_05,gr_than_thres_08,gr_than_thres_10
3,167354641,A1,6.46,36,40000.0,2.63,0.008490,False,False,False
16,167588567,A1,6.46,36,38000.0,2.63,0.079980,True,False,False
43,168101187,A1,6.46,36,20000.0,2.63,0.033295,False,False,False
59,166819641,A4,8.19,60,40000.0,2.42,0.083972,True,False,False
6,167647256,A4,8.19,36,30000.0,2.63,0.076347,True,False,False


[back to top](#toc)

<a id = 'scr'></a>
#### 3.B.b. script


```python
# predict_default.py
import numpy as np
import pandas as pd
import pickle as pickle
import sklearn
import sys

class LoanDefaultModel:
    """A model that predicts loan default rate for loans on LendingClub.com"""

    def __init__(self, model_dict_location):
        with open(model_dict_location, 'rb') as f:
            model_dict = pickle.load(f)
            self.model = model_dict['model']
            self.clean_dict = model_dict['clean_dict']
            self.engineer_dict = model_dict['engineer_dict']
            self.threshold_dict = model_dict['threshold_dict']

    def predict_proba(self, X, clean=True, augment = True):
        X_orig = X.copy()
        if clean:
            X = self.clean_data(X)
        if augment:
            X = self.engineer_features(X)
        return X_orig, X, self.model.predict_proba(X)[:,1]

    def clean_data(self, df):
        clean_dict = self.clean_dict

        df = df.drop(clean_dict['drop_cols_in_open_not_in_master'], axis = 1)
        df = df.drop(clean_dict['drop_cols'], axis = 1)

        df.loc[:, 'grade'] = df.grade.map(clean_dict['grade_map'])
        df.loc[:, 'sub_grade'] = df.sub_grade.apply(lambda x: clean_dict['grade_map'][x[0]]*5 + int(x[1]))
        df.loc[:, 'emp_length'] = df.emp_length.apply(lambda x: clean_dict['el_map'][x] if pd.notnull(x) else 0)
        df.loc[:, 'sec_app_earliest_cr_line'] = pd.to_datetime(df.sec_app_earliest_cr_line, errors='coerce')
        df.loc[:, 'earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, errors = 'coerce')
        df.loc[:, 'list_d'] = pd.to_datetime(df.list_d, errors = 'coerce')
        df.loc[:, 'home_ownership'] = df.home_ownership.replace(['NONE', 'OTHER'], 'ANY')
        df.loc[:, 'purpose'] = df.purpose.apply(lambda x: x.replace(" ", "_").lower())
        try:
            df.loc[:, 'purpose'] = df.purpose.replace(clean_dict['purpose_map'])
        except:
            print("Error with categorical feature purpose:", sys.exc_info()[0])
            raise
        df.loc[:, 'term'] = df.term.replace({36: ' 36 months', 60: ' 60 months'})
        df.loc[:, 'application_type'] = df.application_type.replace({'INDIVIDUAL':'Individual', 'JOINT':'Joint App'})

        df.loc[:, clean_dict['cat_to_num_cols']] = df[clean_dict['cat_to_num_cols']].apply(lambda s: pd.to_numeric(s, errors = 'coerce'))
        df.loc[:, clean_dict['cat_cols']] = df[clean_dict['cat_cols']].fillna('Missing')
        df.loc[:, 'verified_status_joint'] = df.verified_status_joint.replace({' ': 'Missing'})
        df = df.drop(clean_dict['num_cols_miss_sparse'], axis = 1)
        for col in clean_dict['num_cols_miss']:
            df.loc[:, col+'_mflag'] = df[col].isnull()
        for col in clean_dict['num_cols_miss_1200']:
            df.loc[:, col] = df[col].fillna(1200)
        for col in clean_dict['num_cols_miss_median']:
            df.loc[:, col] = df[col].fillna(clean_dict['num_cols_miss_median_values'][col])

        return df


    def engineer_features(self, df):
        engineer_dict = self.engineer_dict

        df.loc[:, 'credit_history'] = ((df.list_d - df.earliest_cr_line)/ np.timedelta64(1, 'D')).astype(int)
        df = df.drop(['earliest_cr_line','list_d'], axis = 1)
        df.loc[:, 'itlm'] = df.annual_inc/df.installment
        df = pd.get_dummies(df)
        df, _ = df.align(engineer_dict['abt_df'], join = 'right', axis = 1, fill_value = 0)

        return df

def main(data_location, model_location, output_location, clean = True, augment = True):

    df = pd.read_csv(data_location)
    default_model = LoanDefaultModel(model_location)

    orig_df, df, pred = default_model.predict_proba(df, clean = clean, augment = augment)
    orig_df.loc[:, 'predict_proba'] = pred
    orig_df.loc[:, 'gr_than_thres_05'] = (orig_df.predict_proba > default_model.threshold_dict['thres_05'])
    orig_df.loc[:, 'gr_than_thres_08'] = (orig_df.predict_proba > default_model.threshold_dict['thres_08'])
    orig_df.loc[:, 'gr_than_thres_10'] = (orig_df.predict_proba > default_model.threshold_dict['thres_10'])
    
    orig_df[['id','sub_grade', 'int_rate','term','loan_amnt','exp_default_rate', 'predict_proba', 
         'gr_than_thres_05', 'gr_than_thres_08', 'gr_than_thres_10']].to_csv(output_location, index = None)

    return

if __name__ == '__main__':
    main(*sys.argv[1:])


```

[back to top](#toc)
<a id = 'exe'></a>
##### 3.B.b.i. To execute python file:

1. create `predict_default.py`
2. locate data, e.g., `open_loan_data.csv`, `model.pkl`
3. in terminal, change to the directory where `default_model` is saved and type: `python predict_default.py data_location output_location model_location True True`, the last two arguments are input values for `clean=, augment=` parameters for the `predict_proba` mothod.
4. predictions will be saved in the directory specified in the above command.

For example, if data, model files are all saved under one directory, we can run the model as follows:

**`python predict_default.py loan_data.csv prediction.csv model.pkl True True`**    

[back to top](#toc)
<a id = 'predict'></a>
##### 3.B.b.ii. Sample predictions

In [12]:
pred_df = pd.read_csv('../preds/predictions_v20200309/prediction.csv')
pred_df.shape

(64, 10)

In [14]:
pred_df.sort_values(['predict_proba'])

,id,sub_grade,int_rate,term,loan_amnt,exp_default_rate,predict_proba,gr_than_thres_05,gr_than_thres_08,gr_than_thres_10
3,167354641,A1,6.46,36,40000.0,2.63,0.008490,False,False,False
43,168101187,A1,6.46,36,20000.0,2.63,0.033295,False,False,False
13,167096305,A5,8.81,36,40000.0,2.63,0.040976,False,False,False
19,167746549,B2,11.02,36,40000.0,5.36,0.042930,False,False,False
60,168078848,B3,11.71,36,33925.0,5.36,0.060919,False,False,False
48,167895294,C5,17.74,36,35000.0,7.96,0.075347,True,False,False
6,167647256,A4,8.19,36,30000.0,2.63,0.076347,True,False,False
16,167588567,A1,6.46,36,38000.0,2.63,0.079980,True,False,False
59,166819641,A4,8.19,60,40000.0,2.42,0.083972,True,False,False
42,166990987,B2,11.02,36,30125.0,5.36,0.087283,True,False,False


In [16]:
pred_df.sort_values(['predict_proba']).mean()

id                  1.676822e+08
int_rate            1.588687e+01
term                3.937500e+01
loan_amnt           2.419844e+04
exp_default_rate    8.015625e+00
predict_proba       2.150898e-01
gr_than_thres_05    9.218750e-01
gr_than_thres_08    7.500000e-01
gr_than_thres_10    7.343750e-01
dtype: float64

**Notes:**
1. The final `predict_default.py` file makes prediction on raw open loan data downloaded directly from Lendingclub.com, it outputs a csv file with predicted probability as well as whehter it meets preselected thresholds.
2. Expected default rate provided by lending clube is pegged to loan's grade, thus does not provide more information.
3. For loans present on 20200309, with preselected thresholds, we can build a **portfolio of 27%, 25%, or 8% of loans with expected default rate of 11%, 9%, or 6%**.
4. Following the prediction, we can invest in chosen loans by finding them by their unique loanID on lendingclub.com.

**End of current notebook**

**[back to top](#toc)**

---

**Prior notebook: [part 3 - model selection + model evaluation](proj-classification-loanDefault-p3-modelSelection-modelEvaluation-max-v2020Feb-remote.ipynb)**

**Delieverables:**

- **[python script](../deliverables/predict_default.py)**

- **[jupyter notebook](../deliverables/predict_default.ipynb)**